# Importing Necessary Libraries

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.utils import to_categorical

# Load the Data

In [2]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

In [3]:
train_df.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Prepare the Training Data

In [4]:
y_train = train_df['label']
X_train = train_df.drop('label', axis=1)
X_train = X_train.values.reshape(-1, 28, 28, 1) / 255.0
y_train = to_categorical(y_train, 10)

# Split the Data into Training and Validation Sets

In [5]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=42)

# Build the CNN Model

In [6]:
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(10, activation='softmax'))

In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 13, 13, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 5, 5, 64)         0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 1600)              0         
                                                                 
 dense (Dense)               (None, 64)                1

# Compile and Train the Model

In [8]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=10, batch_size=64, validation_data=(X_val, y_val))

Epoch 1/10
591/591 [==============================] - 8s 13ms/step - loss: 0.2207 - accuracy: 0.9341 - val_loss: 0.0811 - val_accuracy: 0.9760
Epoch 2/10
591/591 [==============================] - 7s 13ms/step - loss: 0.0645 - accuracy: 0.9803 - val_loss: 0.0524 - val_accuracy: 0.9848
Epoch 3/10
591/591 [==============================] - 9s 15ms/step - loss: 0.0450 - accuracy: 0.9850 - val_loss: 0.0532 - val_accuracy: 0.9843
Epoch 4/10
591/591 [==============================] - 9s 16ms/step - loss: 0.0345 - accuracy: 0.9893 - val_loss: 0.0495 - val_accuracy: 0.9860
Epoch 5/10
591/591 [==============================] - 10s 16ms/step - loss: 0.0267 - accuracy: 0.9917 - val_loss: 0.0366 - val_accuracy: 0.9893
Epoch 6/10
591/591 [==============================] - 9s 15ms/step - loss: 0.0214 - accuracy: 0.9930 - val_loss: 0.0482 - val_accuracy: 0.9867
Epoch 7/10
591/591 [==============================] - 10s 17ms/step - loss: 0.0171 - accuracy: 0.9941 - val_loss: 0.0327 - val_accuracy: 0.99

# Evaluate the Model on Validation Data

In [9]:
val_loss, val_acc = model.evaluate(X_val, y_val)
print(f'Validation Accuracy: {val_acc * 100:.2f}%')

132/132 [==============================] - 1s 4ms/step - loss: 0.0402 - accuracy: 0.9895
Validation Accuracy: 98.95%


# Prepare the Test Data and Make Predictions

In [10]:
X_test = test_df.values.reshape(-1, 28, 28, 1) / 255.0
predictions = model.predict(X_test)

875/875 [==============================] - 3s 3ms/step


# Preparing for Submission

In [11]:
predicted_labels = np.argmax(predictions, axis=1)
submission = pd.DataFrame({'ImageId': np.arange(1, len(X_test) + 1), 'Label': predicted_labels})
submission.to_csv('submission.csv', index=False)

In [12]:
submission

,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,9
4,5,3
...,...,...
27995,27996,9
27996,27997,7
27997,27998,3
27998,27999,9
